# Quality Control

The KPF DRP has several Quality Control (QC) methods that can be run on the L0, 2D, L1, and 2D objects.  The usual application of QC tests is executing them during normal processing in the main recipe.  The QC methods are defined in the ``QCDefinitions`` class in ``modules/quality_control/src/quality_control.py``.  The results of these QC checks are added to the primary headers of kpf objects, which are written to 2D, L1, and L2 FITS files (but not the L0 files, which, with rare exceptions, are not modified after data collection at WMKO).  The FITS header keywords for QC tests produce are defined in :doc:`data_format`.  One can find the QC tests on the command line using a command like 

``fitsheader -e 0 /data/kpf/L2/20230701/KP.20230701.49940.99_L2.fits | grep QC:``

The KPF DRP has several Quality Control (QC) methods that can be run on the L0, 2D, L1, and 2D objects. The QC tests are run during normal processing in the main recipe. The results of these QC checks are added to the primary headers of kpf objects, which are written to 2D, L1, and L2 FITS files (but not the L0 files, which with rare exceptions are not modified after data collection at WMKO). The QC methods are defined in the `QCDefinitions` class in `modules/quality_control/src/quality_control.py`.

In [1]:
import os
from modules.Utils.kpf_parse import get_kpf_data, HeaderParse
import modules.quality_control.src.quality_control as qc
from modules.quality_control.src.quality_control import execute_all_QCs
from IPython.core.display import display, HTML

myQCdef = qc.QCDefinitions()
QC_lists = myQCdef.list_qc_metrics()

# prevent scrolling in the resulting Notebook cell
display(HTML("<style>.output_scroll { height: auto !important; }</style>"))

Quality Control tests for L0:
   Name: not_junk
      Description: File is not in list of junk files.
      Date levels: ['L0', '2D', 'L1', 'L2']
      Date type: int
      Required data products: []
      Spectrum types (applied to): ['all']
      Master types (applied to): ['all']
      Keyword: NOTJUNK
      Keyword fail value: 0
      Comment: QC: Not in list of junk files
      Database column: None

   Name: L0_data_products
      Description: Expected L0 data products present with non-zero array sizes.
      Date levels: ['L0']
      Date type: int
      Required data products: []
      Spectrum types (applied to): ['all']
      Master types (applied to): ['all']
      Keyword: DATAPRL0
      Keyword fail value: 0
      Comment: QC: L0 data present
      Database column: None

   Name: L0_header_keywords_present
      Description: Expected L0 header keywords present.
      Date levels: ['L0']
      Date type: int
      Required data products: []
      Spectrum types (applied to)

The method `test_all_QCs(kpf_object, data_level)` runs the QC tests on a kpf_object for the stated data_level.

In [2]:
ObsID = 'KP.20230701.49940.99' # 185144 (stellar observation)
data_level = 'L2'
if os.path.isfile(get_kpf_data(ObsID, data_level, return_kpf_object=False)):
    print(f'\033[1m{data_level} QC tests on {ObsID}\033[0m')
    print(f'\033[1m***********************************\033[0m')
    kpf_object = get_kpf_data(ObsID, data_level)
    junk = execute_all_QCs(kpf_object, data_level)

L2 QC tests on KP.20230701.49940.99
***********************************
INFO: Spectrum type: Star
INFO: Running QC: not_junk (NOTJUNK; File is not in list of junk files.)
INFO: QC result: True (True = pass)
INFO: Running QC: L2_datetime (TIMCHKL2; Timing consistency in L2 files.)
INFO: QC result: True (True = pass)
INFO: Running QC: data_L2 (DATAPRL2; All data present in L2.)
INFO: QC result: True (True = pass)
INFO: Running QC: add_kpfera (KPFERA; Not a QC test; used to add the KPFERA keyword to header.)
INFO: The datetime of ObsID is 2023-07-01 13:52:20.
INFO: Result: KPFERA=1.0


Now loop over data levels and obserations

In [3]:
# Define data levels to loop over
data_levels = ['L0', '2D', 'L1', 'L2']

# Define list of observations to loop over - one of every spectrum type
ObsIDs = [
          'KP.20230701.49940.99', # 185144 (stellar observation)
          'KP.20240525.77699.94', # Socal
          'KP.20240528.04120.26', # autocal-bias
          'KP.20240528.08502.04', # autocal-dark
          'KP.20240527.84455.02', # autocal-flat-all
          'KP.20240526.11989.38', # autocal-lfc-all-eve
          'KP.20240528.07447.61', # autocal-thar-all-eve
          'KP.20240527.11183.00', # autocal-une-all-eve
          'KP.20240528.05681.26', # autocal-etalon-all-eve
          'KP.20240527.51851.54', # autocal-etalon-all-night
          'KP.20240528.07772.51', # autocal-thar-hk
         ]

# Run QC tests
for ObsID in ObsIDs:
    for data_level in data_levels:
        if os.path.isfile(get_kpf_data(ObsID, data_level, return_kpf_object=False)):
            print(f'\033[1m{data_level} QC tests on {ObsID}\033[0m')
            print(f'\033[1m***********************************\033[0m')
            kpf_object = get_kpf_data(ObsID, data_level)
            junk = execute_all_QCs(kpf_object, data_level)
            print()
            
# prevent scrolling in the resulting Notebook cell
display(HTML("<style>.output_scroll { height: auto !important; }</style>"))

L0 QC tests on KP.20230701.49940.99
***********************************
INFO: Spectrum type: Star
INFO: Running QC: not_junk (NOTJUNK; File is not in list of junk files.)
INFO: QC result: True (True = pass)
INFO: Running QC: L0_data_products (DATAPRL0; Expected L0 data products present with non-zero array sizes.)
INFO: Method L0_data_products does not exist in qc_obj or another AttributeError occurred: 'KPF0' object has no attribute 'GUIDER_AVG'
INFO: Running QC: L0_header_keywords_present (KWRDPRL0; Expected L0 header keywords present.)
INFO: QC result: True (True = pass)
INFO: Running QC: L0_datetime (TIMCHKL0; Timing consistency in L0 header keywords and ExpMeter table.)
INFO: Method L0_datetime does not exist in qc_obj or another AttributeError occurred: 'KPF0' object has no attribute 'GUIDER_AVG'
INFO: Method exposure_meter_not_saturated does not exist in qc_obj or another AttributeError occurred: 'Header' object has no attribute 'header'
INFO: Method exposure_meter_flux_not_negat

INFO: Spectrum type: Sun
INFO: Running QC: not_junk (NOTJUNK; File is not in list of junk files.)
INFO: QC result: True (True = pass)
INFO: Running QC: L2_datetime (TIMCHKL2; Timing consistency in L2 files.)
INFO: QC result: True (True = pass)
INFO: Running QC: data_L2 (DATAPRL2; All data present in L2.)
INFO: QC result: True (True = pass)
INFO: Running QC: add_kpfera (KPFERA; Not a QC test; used to add the KPFERA keyword to header.)
INFO: The datetime of ObsID is 2024-05-25 21:34:59.
INFO: Result: KPFERA=2.0

L0 QC tests on KP.20240528.04120.26
***********************************
INFO: Spectrum type: Bias
INFO: Running QC: not_junk (NOTJUNK; File is not in list of junk files.)
INFO: QC result: True (True = pass)
INFO: Running QC: L0_data_products (DATAPRL0; Expected L0 data products present with non-zero array sizes.)
INFO: QC result: True (True = pass)
INFO: Running QC: L0_header_keywords_present (KWRDPRL0; Expected L0 header keywords present.)
INFO: QC result: True (True = pass)
INF

INFO: Spectrum type: Flat
INFO: Running QC: not_junk (NOTJUNK; File is not in list of junk files.)
INFO: QC result: True (True = pass)
INFO: Not running QC: D2_lfc_flux (LFC frame that goes into a master has sufficient flux) because Flat not in list of spectrum types: ['LFC']
INFO: Not running QC: data_2D_bias_low_flux (Flux is low in bias exposure.) because Flat not in list of spectrum types: ['Bias']
INFO: Not running QC: data_2D_dark_low_flux (Flux is low in dark exposure.) because Flat not in list of spectrum types: ['Dark']
INFO: Method data_2D_CaHK does not exist in qc_obj or another AttributeError occurred: 'Header' object has no attribute 'header'
INFO: Running QC: data_2D_red_green (DATAPR2D; Red/Green CCD data present with expected array sizes.)
INFO: QC result: True (True = pass)
INFO: Running QC: positive_2D_SNR (POS2DSNR; Red/Green CCD data/var^0.5 not significantly negative.)
INFO: QC result: True (True = pass)
INFO: Running QC: add_kpfera (KPFERA; Not a QC test; used to 

INFO: Spectrum type: ThAr
INFO: Running QC: not_junk (NOTJUNK; File is not in list of junk files.)
INFO: QC result: True (True = pass)
INFO: Not running QC: D2_lfc_flux (LFC frame that goes into a master has sufficient flux) because ThAr not in list of spectrum types: ['LFC']
INFO: Not running QC: data_2D_bias_low_flux (Flux is low in bias exposure.) because ThAr not in list of spectrum types: ['Bias']
INFO: Not running QC: data_2D_dark_low_flux (Flux is low in dark exposure.) because ThAr not in list of spectrum types: ['Dark']
INFO: Method data_2D_CaHK does not exist in qc_obj or another AttributeError occurred: 'Header' object has no attribute 'header'
INFO: Running QC: data_2D_red_green (DATAPR2D; Red/Green CCD data present with expected array sizes.)
INFO: QC result: True (True = pass)
INFO: Running QC: positive_2D_SNR (POS2DSNR; Red/Green CCD data/var^0.5 not significantly negative.)
INFO: QC result: True (True = pass)
INFO: Running QC: add_kpfera (KPFERA; Not a QC test; used to 

INFO: Spectrum type: Etalon
INFO: Running QC: not_junk (NOTJUNK; File is not in list of junk files.)
INFO: QC result: True (True = pass)
INFO: Not running QC: D2_lfc_flux (LFC frame that goes into a master has sufficient flux) because Etalon not in list of spectrum types: ['LFC']
INFO: Not running QC: data_2D_bias_low_flux (Flux is low in bias exposure.) because Etalon not in list of spectrum types: ['Bias']
INFO: Not running QC: data_2D_dark_low_flux (Flux is low in dark exposure.) because Etalon not in list of spectrum types: ['Dark']
INFO: Method data_2D_CaHK does not exist in qc_obj or another AttributeError occurred: 'Header' object has no attribute 'header'
INFO: Running QC: data_2D_red_green (DATAPR2D; Red/Green CCD data present with expected array sizes.)
INFO: QC result: True (True = pass)
INFO: Running QC: positive_2D_SNR (POS2DSNR; Red/Green CCD data/var^0.5 not significantly negative.)
INFO: QC result: True (True = pass)
INFO: Running QC: add_kpfera (KPFERA; Not a QC test; 

INFO: Spectrum type: 
INFO: Running QC: not_junk (NOTJUNK; File is not in list of junk files.)
INFO: QC result: True (True = pass)
INFO: Not running QC: D2_lfc_flux (LFC frame that goes into a master has sufficient flux) because  not in list of spectrum types: ['LFC']
INFO: Not running QC: data_2D_bias_low_flux (Flux is low in bias exposure.) because  not in list of spectrum types: ['Bias']
INFO: Not running QC: data_2D_dark_low_flux (Flux is low in dark exposure.) because  not in list of spectrum types: ['Dark']
INFO: Method data_2D_CaHK does not exist in qc_obj or another AttributeError occurred: 'Header' object has no attribute 'header'
INFO: Running QC: data_2D_red_green (DATAPR2D; Red/Green CCD data present with expected array sizes.)
INFO: QC result: True (True = pass)
INFO: Running QC: positive_2D_SNR (POS2DSNR; Red/Green CCD data/var^0.5 not significantly negative.)
INFO: QC result: True (True = pass)
INFO: Running QC: add_kpfera (KPFERA; Not a QC test; used to add the KPFERA k